# Azure音频低语（预览）示例该示例展示了如何使用Azure OpenAI Whisper模型来转录音频文件。

## 设置首先，我们安装必要的依赖项并导入我们将使用的库。

In [ ]:
! pip install "openai>=1.0.0,<2.0.0"! pip install python-dotenv

In [ ]:
import osimport openaiimport dotenvdotenv.load_dotenv()

### 认证Azure OpenAI 服务支持多种认证机制，包括 API 密钥和 Azure Active Directory 令牌凭据。

In [4]:
use_azure_active_directory = False  # 将此标志设置为 True，如果您正在使用 Azure Active Directory。

#### 使用API密钥进行身份验证要设置OpenAI SDK以使用*Azure API密钥*，我们需要将`api_key`设置为与您的端点关联的密钥（您可以在[Azure门户](https://portal.azure.com)的*"资源管理"*下的*"密钥和端点"*中找到此密钥）。您还将在此处找到您资源的端点。

In [5]:
if not use_azure_active_directory:    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]    api_key = os.environ["AZURE_OPENAI_API_KEY"]    client = openai.AzureOpenAI(        azure_endpoint=endpoint,        api_key=api_key,        api_version="2023-09-01-preview"    )

#### 使用Azure Active Directory进行身份验证现在让我们看看如何通过Azure Active Directory进行身份验证。我们将从安装`azure-identity`库开始。该库将提供我们需要进行身份验证的令牌凭据，并通过`get_bearer_token_provider`辅助函数帮助我们构建一个令牌凭据提供程序。建议使用`get_bearer_token_provider`而不是向`AzureOpenAI`提供静态令牌，因为这个API会自动为您缓存和刷新令牌。有关如何设置Azure Active Directory身份验证与Azure OpenAI的更多信息，请参阅[文档](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)。

In [ ]:
! pip install "azure-identity>=1.15.0"

In [5]:
from azure.identity import DefaultAzureCredential, get_bearer_token_providerif use_azure_active_directory:    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]    api_key = os.environ["AZURE_OPENAI_API_KEY"]    client = openai.AzureOpenAI(        azure_endpoint=endpoint,        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),        api_version="2023-09-01-preview"    )

> 注意：如果未提供以下参数，则AzureOpenAI将从其对应的环境变量中推断出来：- `api_key` 从 `AZURE_OPENAI_API_KEY`- `azure_ad_token` 从 `AZURE_OPENAI_AD_TOKEN`- `api_version` 从 `OPENAI_API_VERSION`- `azure_endpoint` 从 `AZURE_OPENAI_ENDPOINT`

## 部署在本节中，我们将使用`whisper-1`模型创建一个部署，用于转录音频文件。

### 部署：在Azure OpenAI Studio中创建让我们部署一个模型以供whisper使用。前往 https://portal.azure.com，找到您的Azure OpenAI资源，然后导航到Azure OpenAI Studio。点击“部署”选项卡，然后为您想要用于whisper的模型创建一个部署。您为模型提供的部署名称将在下面的代码中使用。

In [6]:
deployment = "whisper-deployment" # 在此处填写从门户获取的部署名称

## 音频转录音频转录，或称为语音转文本，是将口语转换为文本的过程。使用`openai.Audio.transcribe`方法将音频文件流转录为文本。您可以从[GitHub上的Azure AI Speech SDK存储库](https://github.com/Azure-Samples/cognitive-services-speech-sdk/tree/master/sampledata/audiofiles)获取示例音频文件。

In [7]:
# 下载示例音频文件import requestssample_audio_url = "https://github.com/Azure-Samples/cognitive-services-speech-sdk/raw/master/sampledata/audiofiles/wikipediaOcelot.wav"audio_file = requests.get(sample_audio_url)with open("wikipediaOcelot.wav", "wb") as f:    f.write(audio_file.content)

In [ ]:
transcription = client.audio.transcriptions.create(    file=open("wikipediaOcelot.wav", "rb"),    model=deployment,)print(transcription.text)